In [1]:
import numpy as np
import pandas as pd
import pycountry
import glob

In [2]:
path = '/data/users/kgruber/other-data/impacts_paper/'
results_path = path + 'automated-quality-check-round2/'
review_path = path + 'review_rounds/'
output_path = path + 'after-automated-quality-check-round2/'

select file names

- if there is more than 1 file, select xlsx, not csv
- -> always the longest file name

In [3]:
fs = glob.glob(results_path + '*results*')
names = [f[76:79] for f in fs]
lens = [len(f) for f in fs]
d = pd.Series(fs,index=[np.array(names),np.array(lens)])
results_files = d.loc[pd.Series(lens,index=names).groupby(names).max().reset_index().apply(tuple,axis=1)].values

fs = glob.glob(results_path + '*result_*')
names = [f[76:79] for f in fs]
lens = [len(f) for f in fs]
d = pd.Series(fs,index=[np.array(names),np.array(lens)])
results_check_files = d.loc[pd.Series(lens,index=names).groupby(names).max().reset_index().apply(tuple,axis=1)].values

read and merge files from those lists of files

In [4]:
for file in results_files:
    if file[-1] == 'x':
        f = pd.read_excel(file,engine='openpyxl',index_col=0)
    else:
        f = pd.read_csv(file,index_col=0)
    f = f[~f.index.isna()]
    if 'responses' in globals():
        responses = pd.concat([responses,f],axis=0)
    else:
        responses = f

In [5]:
for file in results_check_files:
    if file[-1] == 'x':
        f = pd.read_excel(file,engine='openpyxl',index_col=0)
    else:
        try:
            f = pd.read_csv(file,index_col=0)
        except:
            f = pd.read_csv(file,index_col=0,encoding='unicode_escape')
    f = f[~f.index.isna()]
    if 'results_check' in globals():
        results_check = pd.concat([results_check,f],axis=0)
    else:
        results_check = f

## entry has been individually checked

In [6]:
individual_check = responses['Checked'].replace(1,'Passed').replace(np.nan,'Failed')
individual_check.name = 'individual_check'

## iso codes

test passes if code is iso code or "-"

In [7]:
iso_codes = [list(pycountry.countries)[i].alpha_2 for i in range(len(list(pycountry.countries)))]

In [8]:
def check_iso(code):
    if (code in iso_codes) or (code =='-'):
        return 'Passed'
    else:
        return 'Failed'

In [9]:
is_iso_code = pd.Series(responses['Country:'].apply(check_iso),name='iso_code')

check if there are fails due to spaces

In [10]:
(responses['Country:'][is_iso_code=='Failed'].str.replace(' ','').apply(check_iso)=='Passed').sum()

0

## numeric values

In [11]:
def check_float(number):
    try:
        if np.isnan(float(number)):
            return 'Failed'
        else:
            return 'Passed'
    except:
        return 'Failed'

In [12]:
value_is_numeric = pd.Series(responses['4a. Numeric value, e.g., 0.3, 1.5, 3'].apply(check_float),name='numeric_value')

## Unit match metrics

### prepare unit columns

#### merge columns

In [13]:
def merge_col(line):
    return line.dropna().values

In [14]:
measurement_unit = responses[['4c-1. Measurement unit:','4c-2. Measurement unit:']].apply(merge_col,axis=1)

there are two entries where there are values in both columns

In [15]:
(measurement_unit.apply(len)!=1).sum()

2

In [16]:
measurement_unit[(measurement_unit.apply(len)!=1)]

0.0          [DC, MW/ha]
1.0    [DC, MWh/year/ha]
dtype: object

only use second value, first value is referring to current (DC)

In [17]:
measurement_unit[measurement_unit.apply(len)!=1] = measurement_unit[measurement_unit.apply(len)!=1].apply(lambda x: [x[1]])

In [18]:
measurement_unit = measurement_unit.apply(lambda x: x[0])

#### unify units - replace similar units with one writing

In [19]:
replace = pd.DataFrame({'replace':['acres/MW', 'm²/GWh', 'ha/GWh/y',    'W m-2', 'W/m²', 'WP/m2', 'W_p/m²', 'We m-2', 'We m−2', 'We/m²', 'W_e/m2', 'w/ft2', 'kW_e/m²', 'MWi km−2', 'GW_e/m2', 'kWh/year/m²', 'kWh/m²year',  'kWh/m2/year', 'kWh/m²/year', 'MWh/year/m²', 'GWh/yr/m2',   'GJ/m2/year', 'rho_e W_e/m2', 'm^2', 'm2/VPM (Vehicl mile traveled)'],
                        'with':   ['acre/MW',  'm2/GWh', 'ha/GWh/year', 'W/m2',  'W/m2', 'Wp/m2', 'Wp/m2',  'We/m2',  'We/m2',  'We/m2', 'We/m2',  'W/ft2', 'kWe/m2',  'MWi/km2', 'GWe/m2',   'kWh/year/m2', 'kWh/year/m2', 'kWh/year/m2', 'kWh/year/m2', 'MWh/year/m2', 'GWh/year/m2', 'GJ/year/m2', 'rhoe We/m2',   'm2',  'm2/VPM (Vehicle mile traveled)']})

In [20]:
for i in range(len(replace)):
    measurement_unit = measurement_unit.replace(replace['replace'][i],replace['with'][i])

### categorise metrics and units

In [21]:
# metrics
power_related_metrics = ['power_density','installed_power_density','output_power_density','power per unit area',
                         'capacity_density']
energy_related_metrics = ['energy_density','surface_performance_ratio','energy_yields',
                          'aperture_specific_net_electrical_output']
land_related_metrics = ['land_use_efficiency','land_requirements','total_impact_area','direct_impact_area_permanent',
                        'direct_impact_area_temporary','direct_impact_area','land_transformation','land_use_footprints',
                        'land_use_requirements','area_requirements','direct_land_requirements','land_occupation',
                        'spatial_footprint','land-use intensity','land use intensity',' land use intensity',
                        'land_use_intensity','land_use','land_area','area_required_by_system','land_occupational_value'] # land_occupational_value had to be added
other_metrics = ['land_use per vehicle mile traveled (based on land_use_intensity)','land-use per vehicle mile',
          'land-use impact (total habitat developed)']
# units
footprint_power_related = ['m2/W','m2/Wp','m2/kW','m2/kWp','m2/MW','ha/MW','ha/MWp','acre/MW','acre/MW-DC',
                           'acre/MW-AC','km2/MW']
footprint_energy_related = ['m2/MWh','m2/GWh','km2/GWh','km2/TWh']
footprint_annual_energy_related = ['m2/MWh/year','ha/MWh/year','ha/GWh/year','ha/TWh/year','km2/TWh/year']
power_density = ['W/m2','Wp/m2','We/m2','W/ft2','kW/m2','kWp/m2','kWe/m2','kW/ft2','kW/ha','kW/acre','MW/m2',
                 'MW/ha','MWp/ha','MW/km2','MWi/km2','MW/acre','GWe/m2'] # kW/ft2 had to be added
energy_density = ['Wh/cm2/day','kWh/year/m2','kWh/year/acre','kWh/year/ft2','kWh/year/ha','MWh/year/acre',
                  'MWh/year/m2','MWh/year/ha','GWh/year/m2','GWh/year/km2','GJ/year/m2','TWh/year/km2']
unclear_units = ['MJ/m2','GJ/unit/year','km2 year/GWh','kW/ft','km/GWh','m2/VPM (Vehicle mile traveled)',
                 'm2','rhoe We/m2','MW/h/ha']

In [22]:
metrics_classification = pd.Series(['power']*len(power_related_metrics)+
                                   ['energy']*len(energy_related_metrics)+
                                   ['footprint']*len(land_related_metrics)+
                                   ['other']*len(other_metrics),
                                  index = power_related_metrics+energy_related_metrics+land_related_metrics+other_metrics)
unit_classification = pd.Series(['power']*len(power_density)+
                                ['energy']*len(energy_density)+
                                ['footprint']*(len(footprint_power_related)+len(footprint_energy_related)+len(footprint_annual_energy_related))+
                                ['other']*len(unclear_units),
                                index = power_density+energy_density+footprint_power_related+footprint_energy_related+footprint_annual_energy_related+unclear_units)

check if all metrics and units are included

In [23]:
responses['3. Metrics used:'].apply(lambda x: x in metrics_classification.index.values).mean()

1.0

In [24]:
measurement_unit.apply(lambda x: x in unit_classification.index.values).mean()

1.0

### check units

In [25]:
def check_unit(data):
    metric = metrics_classification[data['Metric']]
    unit = unit_classification[data['Unit']]
    if metric =='other' or unit =='other':
        return 'Unclear'
    if metric ==  unit:
        return 'Passed'
    else:
        return 'Failed'

In [26]:
metric_unit = pd.DataFrame({'Metric':responses['3. Metrics used:'].values,
                            'Unit':measurement_unit.values},
                           index=responses.index)
metric_fits_unit = pd.Series(metric_unit.apply(check_unit,axis=1),name='metric_fits_unit')

In [27]:
metric_unit_classification = pd.DataFrame({'metric_classification':metric_unit.Metric.map(metrics_classification),
                                           'unit_classification':metric_unit.Unit.map(unit_classification)})

In [28]:
metric_fits_unit_classification = pd.concat([metric_fits_unit,metric_unit_classification],axis=1)

## Check range of values and whether Wp (Watt peak) has capacity related power component

### unify units

add combined measurement units as column

In [29]:
responses2  = pd.concat([responses.loc[:,:'4c-2. Measurement unit:'],
                         pd.Series(measurement_unit,name='4c. Measurement unit:'),
                         responses.loc[:,'4d. Type of value':]],axis=1)

### conversion

In [30]:
m2peracre = 4046.86
m2perft2 = 10.7639

WattUnits = pd.DataFrame({'start':    ['W/ft2',   'kW/m2', 'kW/ft2',     'kW/ha','kW/acre',      'MW/m2', 'MW/ha','MW/km2','MW/acre'],
                          'target':   'W/m2',
                          'factor':   [1/m2perft2, 1000,   1000/m2perft2, 0.1,    1000/m2peracre, 1/10**6, 100,    1,       10**6/m2peracre]})
OtherWattUnits = pd.DataFrame({'start':   ['kWp/m2', 'MWp/ha', 'kWe/m2', 'GWe/m2', 'MWi/km2'],
                               'target':  ['Wp/m2',  'Wp/m2',  'We/m2',  'We/m2',  'Wi/m2'],
                               'factor':  [1000,     100,      1000,     10**6,    1]})
WatthUnits = pd.DataFrame({'start':  ['Wh/cm2/day', 'kWh/year/acre', 'kWh/year/ft2','kWh/year/ha','MWh/year/acre','MWh/year/m2','MWh/year/ha','GWh/year/m2','GWh/year/km2','GJ/year/m2','TWh/year/km2'],
                           'target': 'kWh/year/m2',
                           'factor': [3650,         1/m2peracre,     1/m2perft2,    1/10**4,      1000/m2peracre, 1000,         0.1,          10**6,        1,             10**6/3600,  1000]})
AreaPowerUnits = pd.DataFrame({'start':  ['m2/W',  'm2/Wp',  'm2/MW',  'ha/MW', 'ha/MWp', 'acre/MW',      'acre/MW-DC',   'acre/MW-AC',   'km2/MW'],
                               'target': ['m2/kW', 'm2/kWp', 'm2/kW',  'm2/kW', 'm2/kWp', 'm2/kW',        'm2/kW-DC',     'm2/kW-AC',     'm2/kW'],
                               'factor': [1000,    1000,     1/1000,   10,      10,       m2peracre/1000, m2peracre/1000, m2peracre/1000, 1000]})
AreaEnergyUnits = pd.DataFrame({'start':  ['m2/MWh','m2/GWh','km2/GWh','km2/TWh','m2/MWh/year','ha/MWh/year','ha/GWh/year','ha/TWh/year','km2/TWh/year'],
                                'target': ['m2/kWh','m2/kWh','m2/kWh', 'm2/kWh', 'm2/kWh/year','m2/kWh/year','m2/kWh/year','m2/kWh/year','m2/kWh/year'],
                                'factor': [1/1000,  1/10**6, 1,        1/1000,   1/1000,       10,           0.01,         1/10**5,      1]})

unit_conversion = pd.concat([WattUnits,OtherWattUnits,WatthUnits,AreaPowerUnits,AreaEnergyUnits],axis=0)

In [31]:
new_unit = responses2['4c. Measurement unit:'].map(unit_conversion.set_index('start').target)
factor = responses2['4c. Measurement unit:'].map(unit_conversion.set_index('start').factor)
# fill in lines where unit stays the same
new_unit[new_unit.isna()] = responses2['4c. Measurement unit:'][new_unit.isna()]
factor = factor.fillna(1)
new_value = responses2['4a. Numeric value, e.g., 0.3, 1.5, 3'][value_is_numeric=='Passed'].apply(float)*factor[value_is_numeric=='Passed']

In [32]:
responses3 = pd.concat([responses2.loc[:,:'4a. Numeric value, e.g., 0.3, 1.5, 3'],
                        pd.Series(responses2.index.map(new_value),name='4a-1. Converted value',index=responses2.index),
                        responses2.loc[:,'4b. Is power-related component of the land-use requirement expressed as energy e.g., ha/GWh/year?':'4c. Measurement unit:'],
                        pd.Series(new_unit,name='4c-3. Converted measurement unit'),
                        responses2.loc[:,'4d. Type of value':]],axis=1)

categorise power related components for grouping and checking whether where unit is Wp the power component is capacity related

In [33]:
capacity_related_components = ['nameplate (installed) capacity','nameplate (installed) capacity DC','nameplate_capacity',
                               'nominal nameplate capacity','peak_rated_power','peak_capacity',
                               'number of turbines of a particular type']

energy_related_components = ['nameplate capacity multiplied by capacity factor',
                             'nameplate (installed) capacity multiplied by capacity factor',
                             'wind-density multiplied by capacity factor','wind density multiplied by capacity factor',
                             'solar constant/insolation multiplied by capacity factor/efficiency',
                             'typical solar insolation at average-insolation location',
                             'typical solar insolation at high-insolation location',
                             'estimated energy generation (unsure what it means)','modelled energy generation',
                             'simulated using ﬂow-sheet computer program based softbeen simulated using ﬂow-sheet computer program based software Cycle-Tempo',
                             'reported energy generation','net energy generation (electricity generation after substracting energy needed for manufacturing/dismantling, construction/operation, and transportation)',
                             'experimentally measured','annual energy production','net output']

unclear_components = ['unclear','author assumes a typical power per unit area of 2.5\u2009W\u2009m−2',
                      'commercial module output','module','no power component','no power']

In [34]:
powercomponent_classification = pd.Series(['capacity']*len(capacity_related_components)+
                                          ['energy']*len(energy_related_components)+
                                          ['unclear']*len(unclear_components),
                                          index = capacity_related_components+energy_related_components+unclear_components)

classify

In [35]:
powercomponent_class = responses['10. Power-related component of land-use requirements is represented by:'].map(powercomponent_classification)

### check if Wp is only related with capacity related components

In [36]:
def check_Wp_capacity(data):
    if data.new_unit=='Wp/m2':
        if data.powercomp_class=='capacity':
            return 'Passed'
        else:
            return 'Failed'
    else:
        return 'Not applicable'

In [37]:
Wp_is_capacity = pd.Series(pd.DataFrame({'powercomp_class':powercomponent_class,
                                         'new_unit':new_unit}).apply(check_Wp_capacity,axis=1),name='Wp_is_capacity')

### find outliers

only look for outliers where there are more than 10 values -> determine frequency of unit-powercomponentclass combinations

In [38]:
unit_powercomponentclass = responses3['4c-3. Converted measurement unit'] + '_' + powercomponent_class
unit_powercomponentclass[value_is_numeric=='Failed'] = 'non-numeric'
frequency_units = responses3['4c-3. Converted measurement unit'].groupby(unit_powercomponentclass).count()

In [39]:
def find_outliers(data):
    #define a list to accumlate anomalies
    outliers = []
    # Set upper and lower limit to 3 standard deviation
    stdev = np.std(data)
    avg = np.mean(data)
    lower_limit  = avg - stdev*3 
    upper_limit = avg + stdev*3
    # Find outliers
    def test_if_in_range(value):
        return in_range(lower_limit,upper_limit,value)
    return data.apply(test_if_in_range)

def in_range(lower,upper,value):
    if (value > lower) & (value < upper):
        return 'Passed'
    else:
        return 'Failed'

only use numeric values

In [40]:
responses3_numeric = responses3[value_is_numeric=='Passed']
unit_powercomponentclass_numeric = unit_powercomponentclass[value_is_numeric=='Passed']

find outliers within groups of combinations of units and powercomponentclasses larger 10

In [41]:
large_groups = frequency_units.index[(frequency_units>10)]
outliers_per_group = [find_outliers(responses3_numeric[unit_powercomponentclass_numeric==unit]['4a-1. Converted value']) 
                      for unit in large_groups.values]
outliers = pd.concat(outliers_per_group,axis=0)

In [42]:
value_within_range = pd.Series(responses3.index.map(outliers).fillna('Too few values within this group'),
                               name='value_range',index=responses3.index)
value_within_range[value_is_numeric=='Failed'] = 'non-numeric'

In [43]:
value_within_range.groupby(value_within_range).count()

value_range
Failed                                25
Passed                              1797
Too few values within this group     181
non-numeric                           32
Name: value_range, dtype: int64

## check if matches: DOI + first Author + scopus_id + year of publication

In [44]:
rev_file = pd.read_csv(review_path + 'SCOPUS_DOI2.csv',encoding = "utf-8",dtype=str).dropna().drop_duplicates()

In [45]:
mapped_SCOPUS = responses3['1b. DOI link'].map(rev_file.set_index('DOI').SCOPUS)
scopusID_fits_doi = pd.Series(responses3.index.map(mapped_SCOPUS.dropna()==responses3['1a. SCOPUS ID'].apply(str)[mapped_SCOPUS.notna()]),
                              index=responses3.index,name='scopusID_fits_doi'
                             ).fillna('Not available').replace(True,'Passed').replace(False,'Failed')

# merge results

In [46]:
results_quality_check = pd.concat([individual_check,is_iso_code,value_is_numeric,metric_fits_unit,value_within_range,Wp_is_capacity,scopusID_fits_doi],axis=1)

In [47]:
individual_check.groupby(individual_check).count()

individual_check
Passed    2035
Name: individual_check, dtype: int64

In [48]:
is_iso_code.groupby(is_iso_code).count()

iso_code
Failed      74
Passed    1961
Name: iso_code, dtype: int64

In [49]:
value_is_numeric.groupby(value_is_numeric).count()

numeric_value
Failed      32
Passed    2003
Name: numeric_value, dtype: int64

In [50]:
metric_fits_unit.groupby(metric_fits_unit).count()

metric_fits_unit
Failed       40
Passed     1973
Unclear      22
Name: metric_fits_unit, dtype: int64

In [51]:
value_within_range.groupby(value_within_range).count()

value_range
Failed                                25
Passed                              1797
Too few values within this group     181
non-numeric                           32
Name: value_range, dtype: int64

In [52]:
Wp_is_capacity.groupby(Wp_is_capacity).count()

Wp_is_capacity
Failed              13
Not applicable    2013
Passed               9
Name: Wp_is_capacity, dtype: int64

In [53]:
scopusID_fits_doi.groupby(scopusID_fits_doi).count()

scopusID_fits_doi
Not available    1448
Passed            587
Name: scopusID_fits_doi, dtype: int64

# remove passed rows and columns and merge comments

In [54]:
# remove lines where all tests passed or are not available or not applicable
passed_results = (results_quality_check == 'Passed').sum(axis=1)
not_available_results = (results_quality_check == 'Not available').sum(axis=1)
not_applicable_results = (results_quality_check == 'Not applicable').sum(axis=1)
results_quality_check_clean_rows = results_quality_check[passed_results+not_available_results+not_applicable_results<results_quality_check.shape[1]]
# remove columns where all tests passed or are not available
passed_results = (results_quality_check_clean_rows == 'Passed').sum(axis=0)
not_available_results = (results_quality_check_clean_rows == 'Not available').sum(axis=0)
not_applicable_results = (results_quality_check_clean_rows == 'Not applicable').sum(axis=0)
results_quality_check_clean = results_quality_check_clean_rows.loc[:,passed_results+not_available_results+not_applicable_results<results_quality_check_clean_rows.shape[0]]

check if there are no rows with only 'Passed' left

In [55]:
((results_quality_check_clean=='Passed').sum(axis=1)==results_quality_check_clean.shape[1]).sum()

0

In [56]:
for col in results_quality_check_clean.columns:
    if 'merged_results_quality_check_clean' in globals():
        try:
            res = pd.concat([results_quality_check_clean[col],
                             results_check[col+'_comment']],axis=1).dropna(subset=[col])
            merged_results_quality_check_clean = pd.concat([merged_results_quality_check_clean,res],axis=1)
        except:
            res = results_quality_check_clean[col].dropna()
            merged_results_quality_check_clean = pd.concat([merged_results_quality_check_clean,res],axis=1)
    else:
        merged_results_quality_check_clean = pd.concat([results_quality_check_clean[col],
                                                        results_check[col+'_comment']],axis=1).dropna(subset=[col])

In [57]:
comments = pd.DataFrame(np.nan * np.ones(shape=results_quality_check_clean.shape),
                        columns=results_quality_check_clean.columns.values + '_comment_round2',
                        index=results_quality_check_clean.index)

In [58]:
merged_results_quality_check_clean_with_comments = pd.concat([merged_results_quality_check_clean,comments],axis=1)

In [59]:
merged_results_quality_check_clean_with_comments.to_csv(output_path + 'results_automated_quality_check_round2.csv')

In [60]:
responses3.to_csv(output_path + 'results_merged_unified_after_automated_check_round2.csv')

#### notes